In [4]:
import os
import torch 
os.environ['http_proxy'] = 'http://129.183.4.13:8080'
os.environ['https_proxy'] = 'http://129.183.4.13:8080'
os.environ['no_proxy'] = 'yoda,129.183.101.5,172.16.118.13,naboo0,naboo5,nwadmin,172.16.118.137,172.16.118.134'

In [5]:
import netCDF4
from pprint import pprint

with netCDF4.Dataset("/home/research/research/modeling/weather_forecast/3dcorrection/data/raw/data.nc", "r", format="NETCDF4") as file:
    pprint(list(file.variables.keys()))
    pprint(file.dimensions)
    # print(file.dimensions['column'].size)
    print('inputs')
    print('pressure_hl', file['pressure_hl'][:].shape)
    print('sca_inputs', file['sca_inputs'][:].shape)
    print('col_inputs', file['col_inputs'][:].shape)
    print('hl_inputs', file['hl_inputs'][:].shape)
    print('inter_inputs', file['inter_inputs'][:].shape)
    
    print('outputs')
    print('flux_dn_sw', file['flux_dn_sw'][:].shape)
    print('flux_up_sw', file['flux_up_sw'][:].shape)
    print('flux_dn_lw', file['flux_dn_lw'][:].shape)
    print('flux_up_lw', file['flux_up_lw'][:].shape)
    print('hr_sw', file['hr_sw'][:].shape)
    print('hr_lw', file['hr_lw'][:].shape)

['sca_inputs',
 'col_inputs',
 'hl_inputs',
 'pressure_hl',
 'inter_inputs',
 'lat',
 'lon',
 'flux_dn_sw',
 'flux_up_sw',
 'flux_dn_lw',
 'flux_up_lw',
 'hr_sw',
 'hr_lw']
{'col_variable': <class 'netCDF4._netCDF4.Dimension'>: name = 'col_variable', size = 27,
 'column': <class 'netCDF4._netCDF4.Dimension'>: name = 'column', size = 1085440,
 'half_level': <class 'netCDF4._netCDF4.Dimension'>: name = 'half_level', size = 138,
 'hl_variable': <class 'netCDF4._netCDF4.Dimension'>: name = 'hl_variable', size = 2,
 'inter_variable': <class 'netCDF4._netCDF4.Dimension'>: name = 'inter_variable', size = 1,
 'level': <class 'netCDF4._netCDF4.Dimension'>: name = 'level', size = 137,
 'level_interface': <class 'netCDF4._netCDF4.Dimension'>: name = 'level_interface', size = 136,
 'p_variable': <class 'netCDF4._netCDF4.Dimension'>: name = 'p_variable', size = 1,
 'sca_variable': <class 'netCDF4._netCDF4.Dimension'>: name = 'sca_variable', size = 17}
inputs
pressure_hl (1085440, 138, 1)
sca_inputs

In [6]:
import netCDF4
from pprint import pprint

with netCDF4.Dataset("/home/research/research/modeling/weather_forecast/3dcorrection/data/raw/data.nc", "r", format="NETCDF4") as file:
    pressure_hl = torch.tensor(file['pressure_hl'][:])
    sca_inputs = torch.tensor(file['sca_inputs'][:])
    col_inputs = torch.tensor(file['col_inputs'][:])
    hl_inputs = torch.tensor(file['hl_inputs'][:])
    inter_inputs = torch.tensor(file['inter_inputs'][:])
    
    flux_dn_sw = torch.tensor(file['flux_dn_sw'][:])
    flux_up_sw = torch.tensor(file['flux_up_sw'][:])
    flux_dn_lw = torch.tensor(file['flux_dn_lw'][:])
    flux_up_lw = torch.tensor(file['flux_up_lw'][:])

In [15]:
g, _, _ = pressure_hl.shape

In [16]:
g

1085440

In [7]:
def broadcast_features(tensor):
    t = torch.unsqueeze(tensor, -1)
    t = t.repeat((1, 1, 138))
    t = t.moveaxis(1, -1)
    
    return t

In [8]:
def pad_tensor(tensor):
    return F.pad(tensor, (0, 0, 1, 1, 0, 0))

In [12]:
import torch.nn.functional as F
import numpy as np

x = torch.cat([
    hl_inputs,
    pad_tensor(inter_inputs),
    broadcast_features(sca_inputs)
], dim=-1)

y = torch.cat([
    torch.unsqueeze(flux_dn_sw, -1),
    torch.unsqueeze(flux_up_sw, -1),
    torch.unsqueeze(flux_dn_lw, -1),
    torch.unsqueeze(flux_up_lw, -1),
], dim=-1)

In [13]:
# def pad_tensor(tensor):
#     feats = tensor.shape[1]
#     n = tensor.shape[0]
    
#     t = np.insert(tensor, (0, 0, 0), np.zeros((n, feats, 1)), axis=0)
#     t = np.insert(tensor, (n, feats, 0), np.zeros(feats), axis=0)
#     return t

In [149]:
directed_index = np.array([[*range(1, 138)], [*range(137)]])
undirected_index = np.hstack((
    directed_index, 
    directed_index[[1, 0], :]
))
undirected_index = torch.tensor(undirected_index, dtype=torch.long)

In [88]:
x

tensor([1, 2, 3])

In [91]:
y = torch.unsqueeze(x, dim=-1)

In [101]:
y_ = torch.unsqueeze(y, -1)
print(y_.shape)
y_ = y_.repeat((1, 1, 138))
y_ = y_.moveaxis(1, -1)
y_.shape
# y_ = y_.transpose((0, 2, 1))

torch.Size([3, 1, 1])


torch.Size([3, 138, 1])

In [ ]:
sca_inputs

In [ ]:
inter_inputs

In [23]:
a = np.arange(10).reshape((2, -1))
b = (np.arange(10) + 10).reshape((2, -1))
print(a, b)

[[0 1 2 3 4]
 [5 6 7 8 9]] [[10 11 12 13 14]
 [15 16 17 18 19]]


In [34]:
np.insert(a, a.shape[0], np.zeros(a.shape[1]), axis=0)

array([[0, 1, 2, 3, 4],
       [5, 6, 7, 8, 9],
       [0, 0, 0, 0, 0]])

In [81]:
inter_inputs.shape

(1085440, 136, 1)

In [58]:
pad_tensor(inter_inputs)

ValueError: shape mismatch: value array of shape (1085440,136,1) could not be broadcast to indexing result of shape (3,136,1)

In [69]:
np.pad(a, (1, 0), reflect_type='symmetric', constant_values=0)

ValueError: unsupported keyword arguments for mode 'constant': {'reflect_type'}

In [79]:
a[:, :, np.newaxis]

array([[[0],
        [1],
        [2],
        [3],
        [4]],

       [[5],
        [6],
        [7],
        [8],
        [9]]])

In [78]:
np.pad(a[:, :, np.newaxis], [(0, 0), (1, 1), (0, 0)])

array([[[0],
        [0],
        [1],
        [2],
        [3],
        [4],
        [0]],

       [[0],
        [5],
        [6],
        [7],
        [8],
        [9],
        [0]]])

In [14]:
import numpy as np

def concat_over_node(features):
    if len(features.shape) == 2:
        features = features[..., np.newaxis]
        
    for dim in features.shape[-1]:
        up = features[:, 1:, :]
        down = features[:, :137, :]
        np.concatenate()
    return np.concatenate((up, down), axis=2)

In [ ]:
concat_over_nodes()